# Lending club case study

Steps : 

- First load data and get shape
- Drop all coulmns with all value Nan or if column values are same like 0
- Drop columns like URL or cutomer behavior variables which may not have any impact
- Describe dataset
- Drop rows for Current as we are not interested in this :  we are interested in Fully paid and Charged off
- Create derived columns if required  
- Create plots to check for univariate and Bivariate analysis.

Notes: 
identify outliers using box plot.

imp variables: loan_amnt (asked amount by borrower) , funded_amnt (authorized some fund by market place : suggested amount by market place to lender), funded_amnt_inv (actual amount given by lender reaching to borrower),emp_len, check grade and subgrade.

check types of loan :  

conclusion : who may default based on factors?


In [23]:
## imports

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings as ws

In [24]:
ws.filterwarnings('ignore')
pd.options.display.max_columns=150


In [25]:
#load dataset and get shape
loan_data = pd.read_csv('loan/loan.csv')
loan_data.shape

(39717, 111)

// drop columns with null values 

In [26]:
loan_data.dropna(how='all', axis=1, inplace=True)
loan_data.shape

(39717, 57)

drop columns with single values as it will not affect the exploratory analysis

In [27]:
for col in loan_data.columns:
    if len(loan_data[col].unique()) == 1:
        loan_data.drop(col,inplace=True,axis=1)
loan_data.shape

(39717, 51)

###drop customer behavior coulmns as suggested in Upgrad session

In [28]:
drop_col= ['delinq_2yrs','earliest_cr_line','inq_last_6mths','open_acc','pub_rec','revol_bal','revol_util','total_acc','out_prncp','out_prncp_inv','total_pymnt','total_pymnt_inv','total_rec_prncp','total_rec_int','total_rec_late_fee','recoveries','collection_recovery_fee','last_pymnt_d','last_pymnt_amnt','last_credit_pull_d']
loan_data.drop(drop_col, axis=1,inplace=True)
loan_data.shape

(39717, 31)

//describe data

In [29]:
loan_data.describe()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,installment,annual_inc,dti,mths_since_last_delinq,mths_since_last_record,collections_12_mths_ex_med,chargeoff_within_12_mths,pub_rec_bankruptcies,tax_liens
count,3.971700e+04,3.971700e+04,39717.000000,39717.000000,39717.000000,39717.000000,3.971700e+04,39717.000000,14035.000000,2786.000000,39661.0,39661.0,39020.000000,39678.0
mean,6.831319e+05,8.504636e+05,11219.443815,10947.713196,10397.448868,324.561922,6.896893e+04,13.315130,35.900962,69.698134,0.0,0.0,0.043260,0.0
std,2.106941e+05,2.656783e+05,7456.670694,7187.238670,7128.450439,208.874874,6.379377e+04,6.678594,22.020060,43.822529,0.0,0.0,0.204324,0.0
min,5.473400e+04,7.069900e+04,500.000000,500.000000,0.000000,15.690000,4.000000e+03,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0
25%,5.162210e+05,6.667800e+05,5500.000000,5400.000000,5000.000000,167.020000,4.040400e+04,8.170000,18.000000,22.000000,0.0,0.0,0.000000,0.0
50%,6.656650e+05,8.508120e+05,10000.000000,9600.000000,8975.000000,280.220000,5.900000e+04,13.400000,34.000000,90.000000,0.0,0.0,0.000000,0.0
75%,8.377550e+05,1.047339e+06,15000.000000,15000.000000,14400.000000,430.780000,8.230000e+04,18.600000,52.000000,104.000000,0.0,0.0,0.000000,0.0
max,1.077501e+06,1.314167e+06,35000.000000,35000.000000,35000.000000,1305.190000,6.000000e+06,29.990000,120.000000,129.000000,0.0,0.0,2.000000,0.0
